In [33]:
import pandas as pd

def crear_dataframe_actividades():
    """ Crea un DataFrame de Pandas con una lista de actividades inventadas.
    
    Returns:
        DataFrame: Un DataFrame con las columnas 'Actividad', 'Descripción', 'Precedentes', 'Duración'.
    """
    data = {
    'Actividad': [
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21
    ],
    'Descripcion': [
        'Decision preliminar',
        'Determinacion y analisis de objetivos',
        'Sugerencia del plan de estudios',
        'Presentacion de la propuesta',
        'Consulta con el sector academico del departamento de matematicas',
        'Consulta con el sector academico de la UNAM',
        'Consulta con el sector academico de otras instituciones academicas',
        'Consulta con el sector academico de otras instituciones no academicas',
        'Consulta con el sector administrativo de la UNAM',
        'Analisis, discusion e incorporacion de sugerencias del departamento de matematicas',
        'Analisis, discusion e incorporacion de sugerencias de la UNAM',
        'Analisis, discusion e incorporacion de sugerencias de instituciones academicas',
        'Analisis, discusion e incorporacion de sugerencias de instituciones no academicas',
        'Analisis comparativo de planes similares en la UNAM',
        'Adicion de consideraciones nacionales',
        'Adicion de consideraciones internacionales',
        'Elaboracion del documento administrativo',
        'Presentacion en el consejo tecnico de la Facultad de Ciencias',
        'Observaciones del consejo tecnico',
        'Elaboracion del plan de estudios final',
        'Presentacion de propuesta definitiva a consejos tecnico'
    ],
    'Precedentes': [
        '', '1', '2', '3', '4', '4', '4', '4', '4', '5', '6', '7', '8', '11',
        '10, 14', '12, 13, 18', '9, 16', '15', '18', '17, 19', '20'
    ],
    'Duracion': [
        3, 15, 10, 2, 8, 10, 10, 8, 5, 20, 10, 8, 6, 10, 20, 15, 10, 3, 8, 8, 8
    ]
    }

 
    return pd.DataFrame(data)

def guardar_en_excel(dataframe, nombre_archivo):
    """ Guarda el DataFrame en un archivo Excel.
    
    Args:
        dataframe (DataFrame): DataFrame a guardar.
        nombre_archivo (str): Nombre del archivo Excel.
    """
    # Crear un writer de Pandas Excel usando XlsxWriter o openpyxl como motor.
    escribe = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')  # O 'openpyxl' si prefieres
    # Convertir el dataframe a un objeto Excel.
    dataframe.to_excel(escribe, sheet_name='Actividades', index=False)
    # Importante: usar close en lugar de save
    escribe.close()

# Luego procedes con la ejecución del resto de tu código normalmente


# Crear el DataFrame de actividades
df_actividades = crear_dataframe_actividades()

# Guardar el DataFrame en un archivo Excel
nombre_archivo_excel = 'Actividades_Proyecto.xlsx'
guardar_en_excel(df_actividades, nombre_archivo_excel)
print("El archivo Excel ha sido creado y guardado con éxito.")


El archivo Excel ha sido creado y guardado con éxito.


In [26]:
df_actividades

,Actividad,Descripcion,Precedentes,Duracion
0,1,Decision preliminar,,3
1,2,Determinacion y analisis de objetivos,1,15
2,3,Sugerencia del plan de estudios,2,10
3,4,Presentacion de la propuesta,3,2
4,5,Consulta con el sector academico del departame...,4,8
5,6,Consulta con el sector academico de la UNAM,4,10
6,7,Consulta con el sector academico de otras inst...,4,10
7,8,Consulta con el sector academico de otras inst...,4,8
8,9,Consulta con el sector administrativo de la UNAM,4,5
9,10,"Analisis, discusion e incorporacion de sugeren...",5,20


In [29]:
import pandas as pd
import networkx as nx

def leer_datos_excel(archivo):
    """ Lee los datos del proyecto desde un archivo Excel.
    Args:
        archivo (str): Ruta del archivo Excel.

    Returns:
        DataFrame: DataFrame con las columnas 'Actividad', 'Precedentes', 'Duración'.
    """
    return pd.read_excel(archivo)

def construir_grafo(actividades):
    """ Construye un grafo dirigido desde el DataFrame de actividades.
    Args:
        actividades (DataFrame): DataFrame con las columnas 'Actividad', 'Precedentes', 'Duración'.
    
    Returns:
        DiGraph: Un grafo dirigido que representa la red de actividades.
    """
    G = nx.DiGraph()
    # Primero agregamos todos los nodos con sus duraciones
    for _, row in actividades.iterrows():
        G.add_node(row['Actividad'], duracion=row['Duracion'])

    # Luego agregamos los arcos entre los nodos basados en los precedentes
    for _, row in actividades.iterrows():
        if pd.notna(row['Precedentes']):  # Asegurarnos de que hay precedentes
            precedentes = map(int, row['Precedentes'].split(','))  # Convertir a enteros
            for pre in precedentes:
                G.add_edge(pre, row['Actividad'])  # Agregar arco sin peso

    return G

def calcular_camino_critico(G):
    """ Calcula el camino crítico en el grafo de actividades.
    Args:
        G (DiGraph): Grafo de las actividades del proyecto.

    Returns:
        tuple: Tiempo total del proyecto y lista de actividades críticas.
    """
    duraciones = {node: data['duracion'] for node, data in G.nodes(data=True)}
    for node in G.nodes():
        if 'duracion' not in G.nodes[node]:
            G.nodes[node]['duracion'] = 0
    camino_critico = nx.dag_longest_path(G, weight='weight')
    tiempo_total = sum(duraciones[node] for node in camino_critico)
    return tiempo_total, camino_critico

def generar_reporte(tiempo_total, camino_critico, archivo_salida):
    """ Genera un archivo de texto con el tiempo total y las actividades críticas.
    Args:
        tiempo_total (int): Tiempo total del proyecto.
        camino_critico (list): Actividades que forman el camino crítico.
        archivo_salida (str): Ruta del archivo de salida.
    """
    with open(archivo_salida, 'w') as file:
        file.write(f'Tiempo Total del Proyecto: {tiempo_total} minutos\n')
        file.write('Actividades Críticas:\n')
        file.write(', '.join(map(str, camino_critico)))

# Uso del código
archivo_excel = 'Actividades_Proyecto.xlsx'
actividades = leer_datos_excel(archivo_excel)
grafo_actividades = construir_grafo(actividades)
tiempo_total, actividades_criticas = calcular_camino_critico(grafo_actividades)
generar_reporte(tiempo_total, actividades_criticas, 'reporte_proyecto.txt')
 

In [32]:
# Suponiendo que tienes estas variables y funciones definidas anteriormente:
# tiempo_total, actividades_criticas = calcular_camino_critico(grafo_actividades)
# generar_reporte(tiempo_total, actividades_criticas, 'reporte_proyecto.txt')

# Ejemplo de cómo podrías calcular el camino crítico y usar los resultados:
tiempo_total, actividades_criticas = calcular_camino_critico(grafo_actividades)

# Imprimir resultados en la consola
print("Tiempo Total del Proyecto: {} minutos".format(tiempo_total))
print("Actividades Críticas: {}".format(", ".join(map(str, actividades_criticas))))

# Generar el reporte en un archivo de texto
generar_reporte(tiempo_total, actividades_criticas, 'reporte_proyecto.txt')


Tiempo Total del Proyecto: 124 minutos
Actividades Críticas: 1, 2, 3, 4, 6, 11, 14, 15, 18, 16, 17, 20, 21


In [30]:
!cat reporte_proyecto.txt


Tiempo Total del Proyecto: 124 minutos
Actividades Críticas:
1, 2, 3, 4, 6, 11, 14, 15, 18, 16, 17, 20, 21